z = 2x^2 + 3에서 x의 기울기를 계산하는 코드

In [18]:
#! /usr/bin/env python3
import torch

x = torch.tensor(data=[2.0, 3.0], requires_grad=True) # [2.0, 3.0] 모양의 tensor를 생성, requires_grad: grad를 계산할지 여부
y = x**2
z = 2*y + 3

target = torch.tensor([3.0,4.0]) 
loss = torch.sum(torch.abs(z-target)) # 임의의 L1 Norm loss를 만듦
loss.backward()

In [19]:
print(x)
print(y)
# z = 2x^2 + 3 을 계산한 결과
print(z)

tensor([2., 3.], requires_grad=True)
tensor([4., 9.], grad_fn=<PowBackward0>)
tensor([11., 21.], grad_fn=<AddBackward0>)


In [20]:
print(z)

# 그레디언트(기울기)를 계산한 결과 -> 잎노드(leaf nood)가 아니기 때문에
print(x.grad, y.grad, z.grad)

# print(loss)

tensor([11., 21.], grad_fn=<AddBackward0>)
tensor([ 8., 12.]) None None


간단한 선형회귀분석 모델 -> 기울기 계산 및 w, b 업데이크 과정

In [21]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init

In [22]:
# 1. y = 2*x + 3 모양을 따르는 데이터를 numData만큼 생성
# 2. 실제 데이터와 유사할 수 있도록 mean = 0, std = 1을 따르는 noise를 데이터에 더해줌
numData = 1000
numEpoch = 5000

x = init.uniform_(torch.Tensor(numData, 1), -10, 10) # 유니폼하게 Tensor를 init하는데, numData만큼의 수를 -10 ~ 10 범위에서 랜덤하게 초기화하는 듯
noise = init.normal_(torch.FloatTensor(numData, 1), std = 1) # normal하게 초기화하는게 뭘까 아마 정규분포겠지 == 맞네 가우시안 따르는? 평균은 0(defalt), 그리고 std(표준편차) = 1
y = 2*x + 3
yNoise = 2*(x + noise) + 3

In [72]:
model = nn.Linear(1, 1) # 선형 회귀 모델. nn,Linear(들어오는 특성 수, 결과로 나오는 특성 수, 편차 사용 여부), 변수로는 Weight와 bias 가 있음.
# 우리는 input = x(1개의 dimention을 가진 1000개의 data), output = y(x와 동일) -> 따라서 인자로 1, 1을 넣음
lossFunc = nn.L1Loss() # loss 함수로는 L1 loss를 지정.

In [73]:
optimizer = optim.SGD(model.parameters(), lr=0.01) # 최적화 함수를 불러옴. 여기서는 SDG(stochastic gradient descent)(한번에 들어오는 데이터의 수대로 경사하강법 알고리즘을 적용하는 최적화 함수)를 사용.
# 최적화할 변수로 model의 parameters를 전달

# model.parameters >> 두개의 값(weight와 bias)
for p in model.parameters():
    print(p)

Parameter containing:
tensor([[0.2271]], requires_grad=True)
Parameter containing:
tensor([0.3863], requires_grad=True)


In [75]:
label = yNoise
for i in range(numEpoch):
    optimizer.zero_grad() # 각 에포크 반복시 지난번에 계산했던 기울기를 0으로 초기화 하는 optimizer
    output = model(x)

    loss = lossFunc(output, label) # loss함수는 output(model(x)의 출력), label(y = 2*x + 3 함수에 노이즈를 섞어준 것)
    loss.backward() # w, b 에대한 gradient가 계산된
    optimizer.step() # gradient에 대해 lr 만큼 업데이트

    if i % 10 == 0:
        print(loss.data)
        paramList = list(model.parameters())
        print(paramList[0].item(), paramList[1].item())

print(paramList[0].item(), paramList[1].item())

tensor(1.5872)
2.0120065212249756 2.982038974761963
tensor(1.5872)
2.0120303630828857 2.9820590019226074
tensor(1.5872)
2.0120229721069336 2.9820590019226074
tensor(1.5872)
2.0120155811309814 2.9820590019226074
tensor(1.5872)
2.0120081901550293 2.9820590019226074
tensor(1.5872)
2.012000799179077 2.9820590019226074
tensor(1.5872)
2.011993408203125 2.9820590019226074
tensor(1.5872)
2.012023687362671 2.982038974761963
tensor(1.5872)
2.0120162963867188 2.982038974761963
tensor(1.5872)
2.0120089054107666 2.982038974761963
tensor(1.5872)
2.0120015144348145 2.982038974761963
tensor(1.5872)
2.0120253562927246 2.9820590019226074
tensor(1.5872)
2.0120179653167725 2.9820590019226074
tensor(1.5872)
2.0120105743408203 2.9820590019226074
tensor(1.5872)
2.012003183364868 2.9820590019226074
tensor(1.5872)
2.011995792388916 2.9820590019226074
tensor(1.5872)
2.012026071548462 2.982038974761963
tensor(1.5872)
2.0120186805725098 2.982038974761963
tensor(1.5872)
2.0120112895965576 2.982038974761963
tensor(